In this notebook, we are going to have a quick look at a new functionality introduced in `choco-solver-4.10.0`: the Conflict-Driven Clause learning (CDCL) framework.

In [ ]:
%%loadFromPOM
<repository>
  <id>oss-sonatype-snapshots</id>
  <url>https://oss.sonatype.org/content/repositories/snapshots/</url>
</repository>
<dependency>
  <groupId>org.choco-solver</groupId>
  <artifactId>choco-solver</artifactId>
  <version>4.10.0-SNAPSHOT</version>
</dependency>

Such a framework is an adapation of the well-konw SAT [CDCL algorithm](https://en.wikipedia.org/wiki/Conflict-driven_clause_learning) to discrete constraint solver.
By exploiting the implication graph (that records events, i.e. variables' modifications), this algorithm is able to derive a new constraint from the events that led to a contradiction. 

Once added to the constraint network, this constraint makes possible to "backjump" (non-chronological backtrack) to the appropriate decision in the decision path.

In CP, learned constraints are denoted "signed-clauses" which is a disjunction of signed-literals, *i.e.* membership unary constraints : $\bigvee_{i = 0}^{n} X_i \in D_{i}$ where $X_i$ are variables and $D_i$ a set of values.
A signed-clause is satisfied when at least one signed-literal is satisfied. 

#### Warning #### 

> In CP, CDCL algorithm requires that each constraint of a problem can be explained. Even though a default explanation function for any constraint, dedicated functions offers better performances. 
> In `choco-solver-4.10.0` a few set of constraints is equiped with dedicated explanation function (unary constraints, binary and ternary, sum and scalar). 

Ok, now let's move to a small example: the [Latin Square Problem](https://en.wikipedia.org/wiki/Latin_square).
A latin square is an $n \times n$ array filled with $n$ different values, each occurring exactly once in each row and exactly once in each column.

Here is a function that models a naive version of the LS problem:

In [ ]:
import java.util.function.Function;
import static java.util.stream.IntStream.range;
import org.chocosolver.solver.Model;
import org.chocosolver.solver.variables.IntVar;
import org.chocosolver.solver.search.strategy.Search;

Function<Integer, Model> latin = n -> {
    Model model = new Model();
    IntVar[] vars = model.intVarArray("c", n * n, 1, n);
    // Constraints
    for (int i = 0; i < n; i++) {
        int j = i;
        range(0, n - 1)
                .forEach(a -> range(a + 1, n)
                        .forEach(b -> {
                            vars[j * n + a].ne(vars[j * n + b]).post(); // rows
                            vars[a * n + j].ne(vars[b * n + j]).post(); // cols
                        }));
    }
    model.getSolver().setSearch(Search.inputOrderLBSearch(vars));
    model.getSolver().showShortStatistics();
    return model;
};

A solution to LS problem with $n$ = 14 requires 523513 nodes:

In [ ]:
latin.apply(14).getSolver().solve();

Using CDCL, the same solution can be found in only 1187 nodes:

In [ ]:
Model m = latin.apply(14);
m.getSolver().setLearningSignedClauses();
m.getSolver().solve();

Indeed, each of the 907 failures is derived into a new signed-clause which helps reducing the search space.
Since the enumeration strategy is static, so the 2nd search space is ensured to be strictly included in the 1st one. 

#### Remark ####
> The model can be improved by using an `allDifferent` constraint which offers a more powerful filtering algorithm. In that case, the CDCL doest not reduce the number of nodes explored.


## Limitations ##

One may noticed that, in comparison to the no-CDCL approach, very few nodes can be explored per second.
This is due to the application of the CDCL algorithm which globally slows down the resolution process.
Indeed:
1. Events need to be recorded in the implication graph.
2. New operations are executed on each conflict to derive a signed-clauses. 
3. The number of learned signed-clauses increases w.r.t. the number of conflict, this can slow down the propagation step (reaching a fix-point or detecting a new conflict).

The last point is probably the most expensive one. That's why, from time to time, some learned clauses need to be forgotten (in `DefaultSettings`: every 100000 failures, half of them are removed under conditions).
Beware, removing a learned sign may not be inconsequential.
For example, the "same" contradiction can thrown again in the future, or the enumeration strategy may be impacted.

In conclusion, CDCL can be very powerful in reducing search space. 
However, this reduction sometimes does not compensate the algorithm cost it comes with.